# System

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_report_function - Start ----------\n")

In [ ]:
medallion_name   = "Report"
if is_debug: print(f"medallion_name: {medallion_name}")

In [ ]:
# Current session info
if is_debug: display(ss.getActiveSession())

## Print debug info

In [ ]:
if is_debug:
    def fn_print_debug_info(
        status,
        fn_name,
        par
    ):
        print(f"\n----- {status} ----- {fn_name}() (UTC: {dt.datetime.now()}) - Start -----")
        for k, v in par.items(): print(f"{k}: {str(v)}")
        print(f"----- {status} ----- {fn_name} - End -----\n")

        return (True)

# Email

## Create report

In [ ]:
def fn_create_email_report(report_type):
    # report_type:
    #   • "Success" --> alert IN ("Success", "Warning")
    #   • "Error"   --> alert = "Danger"
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        html  = None
        table = None

        if report_type == "Error":
            sql_code = f"""SELECT
    IFNULL(`process_timestamp`  , \"\") AS `process_timestamp`
    , IFNULL(`medallion_name`   , \"\") AS `medallion_name`
    , IFNULL(`source_name`      , \"\") AS `source_name`
    , IFNULL(`locals`           , \"\") AS `locals`
    , IFNULL(`alert`            , \"\") AS `alert`
    , IFNULL(`alert_description`, \"\") AS `alert_description`
    , IFNULL(`logged_datetime`  , \"\") AS `logged_datetime`
FROM delta.`{global_parameter.abfs_path_lh_log}/Tables/log`
WHERE
    `alert`                 = \"Danger\"
    AND `process_timestamp` = \"{global_parameter.process_timestamp}\"
ORDER BY `logged_datetime`;"""
        
        elif report_type == "Success":
            sql_code = f"""SELECT *
FROM
  (
    SELECT
        IFNULL(`process_timestamp`  , \"\") AS `process_timestamp`
        , IFNULL(`medallion_name`   , \"\") AS `medallion_name`
        , IFNULL(`source_name`      , \"\") AS `source_name`
        , IFNULL(`locals`           , \"\") AS `locals`
        , IFNULL(`alert`            , \"\") AS `alert`
        , IFNULL(`alert_description`, \"\") AS `alert_description`
        , IFNULL(`logged_datetime`  , \"\") AS `logged_datetime`
    FROM delta.`{global_parameter.abfs_path_lh_log}/Tables/log`
    WHERE
      `alert`                 = \"Warning\"
      AND `process_timestamp` = \"{global_parameter.process_timestamp}\"

    UNION SELECT
        IFNULL(`process_timestamp`  , \"\") AS `process_timestamp`
        , IFNULL(`medallion_name`   , \"\") AS `medallion_name`
        , IFNULL(`source_name`      , \"\") AS `source_name`
        , IFNULL(`locals`           , \"\") AS `locals`
        , IFNULL(`alert`            , \"\") AS `alert`
        , IFNULL(`alert_description`, \"\") AS `alert_description`
        , IFNULL(`logged_datetime`  , \"\") AS `logged_datetime`
    FROM delta.`{global_parameter.abfs_path_lh_log}/Tables/log`
    WHERE
      `alert`                 = \"Success\"
      AND `source_name`       LIKE \"fn_extract%\"
      AND `source_name`       != \"fn_extract_preparation\"
      AND `process_timestamp` = \"{global_parameter.process_timestamp}\"
  ) AS A
ORDER BY
  CASE
    WHEN A.`alert` = \"Warning\" THEN 1
    WHEN A.`alert` = \"Success\" THEN 2
  END
  , `logged_datetime`;"""

        rv = fn_execute_spark_sql(sql_code)
        sdf         = rv[2]
        sdf_count   = rv[3]

        if sdf_count > 0:
            sdf = sdf.withColumn("dict_parameter", sf.get_json_object(sf.col("locals"),"$.dict_parameter"))
            sdf = sdf.select(
                sf.col("process_timestamp").alias("Timestamp")
                , sf.col("medallion_name").alias("Medallion")
                , sf.col("source_name").alias("Source")
                , sf.col("alert").alias("Alert")
                , sf.col("alert_description").alias("Description")
                , sf.date_format(sf.col("logged_datetime"), "yyyy-MM-dd HH:mm:ss.SSS").alias("Logged")
                , sf.get_json_object(sf.col("dict_parameter"), "$.technology").alias("Technology")
                #, sf.get_json_object(sf.col("dict_parameter"), "$.technology_clean").alias("Technology")
                , sf.get_json_object(sf.col("dict_parameter"), "$.frequency").alias("Frequency")
                , sf.get_json_object(sf.col("dict_parameter"), "$.lakehouse_name_clean").alias("Lakehouse")
                , sf.get_json_object(sf.col("dict_parameter"), "$.server_name").alias("Server")
                , sf.get_json_object(sf.col("dict_parameter"), "$.database_name").alias("Database")
                , sf.get_json_object(sf.col("dict_parameter"), "$.schema_name").alias("Schema")
                , sf.get_json_object(sf.col("dict_parameter"), "$.table_name").alias("Table")
                , sf.get_json_object(sf.col("dict_parameter"), "$.folder_name_clean").alias("Folder")
                , sf.get_json_object(sf.col("dict_parameter"), "$.file_name_clean").alias("File")
                , sf.get_json_object(sf.col("dict_parameter"), "$.worksheet_name").alias("Worksheet")
            )

            # Create HTML table - Start
            html = """<!doctype html>
<html lang="en-US">
    <head>
        <title>ETL Report - """ + global_parameter.now_datetime + """</title>
        <meta charset=\"UTF-8\" />
        <meta name=\"viewport\" content=\"width=device-width, initial-scale=1\" />
        <meta name=\"theme-color\" content=\"#dedede\" />
        <meta name=\"description\" content=\"Report, automatically created as a result of ETL\" />
        <meta name=\"generator\" content=\"Microsoft Fabric\" />
        <meta name=\"author\" content=\"zpl-concept.com\" />
        <style>
            body {
				background-color: #dedede;
			}
            .report_table {
				border-collapse: collapse;
				background-color: #ffffff;
			}
            .report_table th {
				border: 1px solid #333333;
				padding: 5px;
				text-align: center;
				background-color: #7d221d;
				color: #ffffff;
				font-family: Verdana, Arial, Tahoma, Helvetica;
				font-size: 12px;
			}
            .report_table td {
				word-wrap: break-word;
				border: 1px solid #333333;
				padding: 5px;
				font-family: Verdana, Arial, Tahoma, Helvetica;
				font-size: 12px;
                vertical-align: top;
			}
        </style>
    </head>
    <body>
"""

            table = """
        <table class=\"report_table\">
        <tr>
            <th>Timestamp</th>
            <th>Medallion</th>
            <th>Source</th>
            <th>Alert</th>
            <th>Description</th>
            <th>Logged at</th>
            <th>Technology</th>
            <th>Frequency</th>
            <th>Lakehouse</th>
            <th>Server</th>
            <th>Database</th>
            <th>Schema</th>
            <th>Table</th>
            <th>Folder</th>
            <th>File</th>
            <th>Worksheet</th>
        </tr>
"""

            for row in sdf.collect():
                if   row.Alert == "Danger":  tr = "     <tr style=\"background-color: #f8d7da; color: #721c24;\">"
                elif row.Alert == "Warning": tr = "     <tr style=\"background-color: #fff3cd; color: #856404;\">"
                elif row.Alert == "Success": tr = "     <tr style=\"background-color: #d4edda; color: #155724;\">"
                else:                        tr = "     <tr>"

                table += tr + "\
            <td>" + str(row.Timestamp) + "</td>\
            <td>" + str(row.Medallion) + "</td>\
            <td>" + str(row.Source) + "</td>\
            <td>" + str(row.Alert) + "</td>\
            <td>" + str(row.Description) + "</td>\
            <td>" + str(row.Logged) + "</td>\
            <td>" + str(row.Technology) + "</td>\
            <td>" + str(row.Frequency) + "</td>\
            <td>" + str(row.Lakehouse) + "</td>\
            <td>" + str(row.Server) + "</td>\
            <td>" + str(row.Database) + "</td>\
            <td>" + str(row.Schema) + "</td>\
            <td>" + str(row.Table) + "</td>\
            <td>" + str(row.Folder) + "</td>\
            <td>" + str(row.File) + "</td>\
            <td>" + str(row.Worksheet) + "</td>\
        </tr>"

            table = table.replace("<td>None</td>", "<td></td>")

            html += table + """     </table>
    </body>
</html>"""
            # Create HTML table - End
        
        # fn_print_debug_info_123("Success", fn_name, par) # test "except"

        alert             = "Success"
        if sdf_count > 0: alert_description = f"Report created successfully. Rows: {sdf_count}"
        else: alert_description = "Report is empty"

        return (alert, alert_description, html, sdf_count)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
            del par

        del table
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, fn_locals_to_json(locals())[2], alert, alert_description)

# Operation

## Get email report

In [ ]:
def fn_get_email_report():
    fn_name        = stk()[0][3]
    caller_fn_name = stk()[1].function.replace("<module>", "")
    if is_debug: par = {}

    try:
        if bool(global_parameter.send_error_email) == True:
            rv_danger           = fn_create_email_report("Error")
            rv_danger_html      = rv_danger[2]
            rv_danger_row_count = rv_danger[3]

        if bool(global_parameter.send_success_email) == True:
            rv_success           = fn_create_email_report("Success")
            rv_success_html      = rv_success[2]
            rv_success_row_count = rv_success[3]

        output = {}
        
        if bool(global_parameter.send_error_email) == True:
            output["rv_danger_row_count"]  = rv_danger_row_count
            output["rv_danger_html"]       = rv_danger_html
        else:
            output["rv_danger_row_count"]  = 0
            output["rv_danger_html"]       = ""
        
        if bool(global_parameter.send_success_email) == True:
            output["rv_success_row_count"] = rv_success_row_count
            output["rv_success_html"]      = rv_success_html
        else:
            output["rv_success_row_count"] = 0
            output["rv_success_html"]      = ""
        
        output = j.dumps(output)

        alert = "Success"
        alert_description = "Report created successfully"

        return(alert, alert_description, output)
    except Exception as ex:
        alert             = "Danger"
        alert_description = str(ex)

        return (alert, alert_description, None)
    finally:
        if is_debug:
            par["locals"] = locals()
            fn_print_debug_info(alert, fn_name, par)
        
        if "sdf"          in locals(): sdf          = str(sdf)
        if "col_name"     in locals(): col_name     = str(col_name)
        if "replace_with" in locals(): replace_with = str(replace_with)
        if "pdf"          in locals(): pdf          = str(pdf)
        if is_debug       == True:     par          = str(par)        
        fn_local_log_insert(global_parameter.process_timestamp, medallion_name, fn_name, str(locals()), alert, alert_description)

In [ ]:
if is_debug: print("\n---------- This notenook name: nb_report_function - End ----------\n")